# 数据属性
PyG中单个图由torch_geometry.data.Data示例描述，包含以下属性：
- data.x:图节点属性，维度为[num_nodes, features]
- data.edge_index:维度为[2, num_edges]，具体包含两个列表，每个列表对应位置上的数字表示相应节点之间存在边连接
- data.edge_attr:图中边的属性信息，维度[num_edges, features]
- data.y:标签，如果是节点分类任务，维度为[num_nodes, *]，如果是图分类任务，维度为[1,...]
- data.pos:[num_nodes,num_dimensions]节点的位置信息

# 图数据的处理

无向图

<img src="https://pytorch-geometric.readthedocs.io/en/latest/_images/graph.svg">

In [2]:
# 无向图
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[1,0,1,2], # node:0和node:1之间有无向边，node:1和node:2之间有无向边
                           [0,1,2,1]])

x = torch.tensor([[-1, 0], # 节点特征
                  [ 0, 3],
                  [ 1, 2]])

edge_attr = torch.tensor([[1],[1],[2],[2]]) # 对应edge_index中四条边的属性

# 创建一个pyg图
data = Data(x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            smiles="C=C") # 因为存在**kwargs,可以自定义属性比如train等标签，方便索引数据
data    # 这边查看的是维度


Data(x=[3, 2], edge_index=[2, 4], edge_attr=[4, 1], smiles='C=C', y=1)

In [12]:
# 遍历data的属性
for key, item in data:
    print(f'{key} found in data: {item}')
    
print("data.num_nodes:", data.num_nodes)
print("data.num_edges:", data.num_edges)
print("data.num_node_features:", data.num_node_features)
print("data.has_isolated_nodes:", data.has_isolated_nodes()) # 是否有孤立节点
print("data.has_self_loops:", data.has_self_loops()) # 节点是否与自己有边
print("data.is_directed:", data.is_directed()) # 判断是否为有向图

x found in data: tensor([[-1,  0],
        [ 0,  3],
        [ 1,  2]])
edge_index found in data: tensor([[1, 0, 1, 2],
        [0, 1, 2, 1]])
edge_attr found in data: tensor([[1],
        [1],
        [2],
        [2]])
data.num_nodes: 3
data.num_edges: 4
data.num_node_features: 2
data.has_isolated_nodes: False
data.has_self_loops: False
data.is_directed: False


In [13]:
# to GPU
device = torch.device('cuda')
data = data.to(device)

# 公共数据集
MoleculeNet

In [2]:
from torch_geometric.datasets import MoleculeNet
dataset = MoleculeNet(root='./temp_data', name='FreeSolv') # name是取哪一个benchmark

In [7]:
dataset

FreeSolv(642)

In [3]:
print("dataset.num_node_features:",dataset.num_node_features)
dataset[0].y

dataset.num_node_features: 9


tensor([[-11.0100]])

In [7]:
dataset[0].edge_attr

tensor([[ 1,  0,  0],
        [ 1,  0,  0],
        [ 1,  0,  0],
        [ 1,  0,  1],
        [ 1,  0,  0],
        [ 1,  0,  1],
        [ 2,  0,  1],
        [ 1,  0,  1],
        [ 2,  0,  1],
        [ 1,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [ 1,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [12,  0,  1],
        [ 1,  0,  1],
        [ 1,  0,  0],
        [ 1,  0,  0]])

In [11]:
dataset[0].is_directed()

False

In [9]:
dataset[0].y # label

tensor([[-11.0100]])

In [9]:
dataset[0]

Data(x=[13, 9], edge_index=[2, 26], edge_attr=[26, 3], smiles='CN(C)C(=O)c1ccc(cc1)OC', y=[1, 1])

## 批处理Mini-batches
每个图的节点和边的数量不同，该如何同时处理
PyG定义一个超图,即对角矩阵A，其中A1代表graph 1的邻接矩阵
X1是graph 1 的特征矩阵

\begin{split}\mathbf{A} = \begin{bmatrix} \mathbf{A}_1 & & \\ & \ddots & \\ & & \mathbf{A}_n \end{bmatrix}, \qquad \mathbf{X} = \begin{bmatrix} \mathbf{X}_1 \\ \vdots \\ \mathbf{X}_n \end{bmatrix}, \qquad \mathbf{Y} = \begin{bmatrix} \mathbf{Y}_1 \\ \vdots \\ \mathbf{Y}_n \end{bmatrix}\end{split}

In [1]:
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader

dataset = MoleculeNet(root='./temp_data', name='FreeSolv') # name是取哪一个benchmark
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in dataloader:
    print(data)

DataBatch(x=[289, 9], edge_index=[2, 560], edge_attr=[560, 3], smiles=[32], y=[32, 1], batch=[289], ptr=[33])
DataBatch(x=[252, 9], edge_index=[2, 474], edge_attr=[474, 3], smiles=[32], y=[32, 1], batch=[252], ptr=[33])
DataBatch(x=[263, 9], edge_index=[2, 502], edge_attr=[502, 3], smiles=[32], y=[32, 1], batch=[263], ptr=[33])
DataBatch(x=[269, 9], edge_index=[2, 524], edge_attr=[524, 3], smiles=[32], y=[32, 1], batch=[269], ptr=[33])
DataBatch(x=[272, 9], edge_index=[2, 532], edge_attr=[532, 3], smiles=[32], y=[32, 1], batch=[272], ptr=[33])
DataBatch(x=[273, 9], edge_index=[2, 524], edge_attr=[524, 3], smiles=[32], y=[32, 1], batch=[273], ptr=[33])
DataBatch(x=[292, 9], edge_index=[2, 568], edge_attr=[568, 3], smiles=[32], y=[32, 1], batch=[292], ptr=[33])
DataBatch(x=[284, 9], edge_index=[2, 544], edge_attr=[544, 3], smiles=[32], y=[32, 1], batch=[284], ptr=[33])
DataBatch(x=[245, 9], edge_index=[2, 456], edge_attr=[456, 3], smiles=[32], y=[32, 1], batch=[245], ptr=[33])
DataBatch(

一个超图里有32个小图，他们的节点特征都是拼在一起的，是如何区分的
data.batch是一个列向量，他将批处理中的节点映射到各自的图中

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mrow data-mjx-texclass="ORD">
    <mi data-mjx-auto-op="false">batch</mi>
  </mrow>
  <mo>=</mo>
  <msup>
    <mrow data-mjx-texclass="ORD">
      <mrow data-mjx-texclass="INNER">
        <mo data-mjx-texclass="OPEN">[</mo>
        <mtable columnspacing="1em" rowspacing="4pt">
          <mtr>
            <mtd>
              <mn>0</mn>
            </mtd>
            <mtd>
              <mo>&#x22EF;</mo>
            </mtd>
            <mtd>
              <mn>0</mn>
            </mtd>
            <mtd>
              <mn>1</mn>
            </mtd>
            <mtd>
              <mo>&#x22EF;</mo>
            </mtd>
            <mtd>
              <mi>n</mi>
              <mo>&#x2212;</mo>
              <mn>2</mn>
            </mtd>
            <mtd>
              <mi>n</mi>
              <mo>&#x2212;</mo>
              <mn>1</mn>
            </mtd>
            <mtd>
              <mo>&#x22EF;</mo>
            </mtd>
            <mtd>
              <mi>n</mi>
              <mo>&#x2212;</mo>
              <mn>1</mn>
            </mtd>
          </mtr>
        </mtable>
        <mo data-mjx-texclass="CLOSE">]</mo>
      </mrow>
    </mrow>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="normal">&#x22A4;</mi>
    </mrow>
  </msup>
</math>

可以通过安装`torch_scatter`对使用`batch`索引对节点特征进行数据聚合
```bash
conda install pytorch-scatter -c pyg
```

<img src="https://raw.githubusercontent.com/rusty1s/pytorch_scatter/master/docs/source/_figures/add.svg?sanitize=true">

In [3]:
import torch
from torch_scatter import scatter

src = torch.randn(10,6,64)
index = torch.tensor([0,1,0,1,2,1])

# Broadcasting in the first and last dim.
out = scatter(src, index, dim=1, reduce='sum')
out.size()

torch.Size([10, 3, 64])

In [10]:
for data in dataloader:
    print(data)
    print(data.edge_index)
    print(data.num_graphs)
    print(data.is_directed())
    x = scatter(data.x, data.batch, dim=0, reduce='mean')
    print(x.size())
    print('-'*100)

DataBatch(x=[310, 9], edge_index=[2, 618], edge_attr=[618, 3], smiles=[32], y=[32, 1], batch=[310], ptr=[33])
tensor([[  0,   1,   1,  ..., 308, 308, 309],
        [  1,   0,   2,  ..., 303, 307, 306]])
32
False
torch.Size([32, 9])
----------------------------------------------------------------------------------------------------
DataBatch(x=[245, 9], edge_index=[2, 460], edge_attr=[460, 3], smiles=[32], y=[32, 1], batch=[245], ptr=[33])
tensor([[  0,   1,   1,   2,   2,   3,   3,   4,   4,   5,   6,   7,   7,   7,
           8,   8,   9,   9,   9,  10,  11,  11,  11,  12,  12,  13,  13,  14,
          14,  14,  15,  16,  17,  18,  18,  19,  19,  20,  20,  20,  21,  21,
          22,  23,  24,  25,  25,  25,  26,  27,  28,  28,  29,  29,  30,  30,
          31,  31,  31,  32,  32,  32,  33,  33,  34,  34,  34,  35,  36,  36,
          36,  37,  37,  37,  38,  38,  39,  39,  40,  40,  40,  41,  41,  41,
          42,  42,  42,  43,  44,  45,  46,  47,  47,  48,  48,  49,  49,  49,
    

## 数据转换
- pre_transform函数
  
  pretransform函数可以看做是对整个数据集进行的预处理，比如对所有的节点进行一个固定的特征映射，或者对所有的图进行归一化。因为这个函数只会在读取数据集时调用一次，所以它通常会在数据集读入时完成一些耗时的操作，例如节点特征的归一化。pre.transform数接受一个Data对象作为参数，该对象表示一个包含节点和边特征的图。函数可以对这个Data对象进行任意的操作，包括添加、删除、修改节点和边的特征。注意，pre_transform函数的操作只会影响整个数据集，在数据集中的每个图都会被应用。
- transform函数
  
  ransform函数是一个更加灵活的函数，它可以对每个图进行不同的转换，例如根据图的特征进行不同的操作。因为transfom函数会在每次数据集被调用时进行，所以它通常会完成一些轻量级的操作，例如为节点特征添加噪声或者对边特征进行采样。transform函数接受一个Data对象作为参数，该对象表示一个包含节点和边特征的图。与pre.transform函数类似，transform函数也可以对这个Data对象进行任意的操作，包括添加、删除、修改节点和边的特征，但不同的是，tansform函数的操作只会影响当前调用的图，而不是整个数据集。
  
综上所述，pre_transform函数和transfom函数的主要区别在于它们被调用的时机和影响范围。pretransform函数只会在读取数据集时调用次，影响整个数据集中的每个图;而transform函数会在每次数据集被调用时调用，只会影响当前调用的图。

In [12]:
def example_function(**kwargs):
    data = Data(x=x,
            edge_index=edge_index,
            edge_attr=edge_attr,
            smiles="C=C",
            **kwargs)
    print(data)
# 调用函数，传递关键字参数
example_function(**{"a":1,"b":2})


Data(x=[3, 2], edge_index=[2, 4], edge_attr=[4, 1], smiles='C=C', a=1, b=2)
